In [1]:
import CropModels
from CropDataset import MyDataSet,normalize_torch,normalize_05,normalize_dataset,preprocess,preprocess_hflip,preprocess_with_augmentation
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from tensorboardX import SummaryWriter
import datetime
import os
import torch
from torch.autograd import Variable
from utils import RunningMean
import utils
from PIL import Image
import numpy as np
import random
NB_CLASS=59
SEED=888
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["CUDA_VISIBLE_DEVICES"] = '3'
torch.backends.cudnn.benchmark = True

In [2]:
BATCH_SIZE=32
IMAGE_SIZE=299    # 不同模型修改不同的Size
IMAGE_TRAIN_PRE='../data/AgriculturalDisease_trainingset/images/'
ANNOTATION_TRAIN='../data/AgriculturalDisease_trainingset/AgriculturalDisease_train_annotations_deleteNoise.json' #是否需要剔除两类异常类
IMAGE_VAL_PRE='../data/AgriculturalDisease_validationset/images/'
ANNOTATION_VAL='../data/AgriculturalDisease_validationset/AgriculturalDisease_validation_annotations_deleteNoise.json' #是否需要剔除两类异常类
date=str(datetime.date.today())
with open(ANNOTATION_TRAIN) as datafile1:
    trainDataFram=pd.read_json(datafile1,orient='records')
with open(ANNOTATION_VAL) as datafile2: #first check if it's a valid json file or not
    validateDataFram =pd.read_json(datafile2,orient='records')    

In [3]:
def getmodel():
    print('[+] loading model... ', end='', flush=True)
    model=CropModels.InceptionV3Finetune(NB_CLASS)
    model.cuda()
    print('Done')
    return model
def train(epochNum):
    writer=SummaryWriter('log/'+date+'/InceptionV3/') # 创建 /log/日期/InceptionResnet的组织形式  不同模型需要修改不同名称
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_05,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_05,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    weight=torch.Tensor([1,3,3,3,3,4,2,3,3,3,3,3,3,3,3,3,2,3,3,3,2,3,4,2,3,1,1,3,2,2,1,3,3,1,3,2,3,3,3,3,2,1,3,2,3,3,3,1,3,3,4,4,3,2,2,3,1,1,3]).cuda()
    criterion=nn.CrossEntropyLoss(weight=weight).cuda()
#     lx, px = utils.predict(model,val_dataLoader)
#     min_loss = criterion(Variable(px), Variable(lx)).item()
    min_loss=4.1
    print('min_loss is :%f'%(min_loss))
    min_acc=0.80
    patience=0
    lr=0.0
    momentum=0.9
    for epoch in range(epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if epoch==3 or epoch==4 or epoch==5:
            lr=0.00006
            momentum=0.95
            print('set lr=:%f,momentum=%f'%(lr,momentum))
        if epoch==6:
            lr=1e-4
            momentum=0.9
            print('set lr=:%f,momentum=%f'%(lr,momentum))        
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/InceptionV3/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/5
            print('loss has increased lr divide 10 lr now is :%f'%(lr))
        if epoch==0 or epoch==1 or epoch==2: #第一轮首先训练全连接层
            lr=1e-3
            optimizer = torch.optim.Adam(model.fresh_params(),lr = lr,amsgrad=True,weight_decay=1e-4)
#             optimizer=torch.optim.SGD(params=model.fresh_params(),lr=lr,momentum=0.9)
            #optimizer=torch.optim.SGD(params=model.fresh_params(),lr=lr,momentum=0.9)
        else:
            optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
             #optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
#             optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))       
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'InceptionV3', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'InceptionV3', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
train(60)

[+] loading model... Done
min_loss is :4.100000
Epoch 0/59
----------
[epoch:0,batch:29]:acc: 0.079167,loss:4.044367
[epoch:0,batch:59]:acc: 0.172917,loss:3.756970
[epoch:0,batch:89]:acc: 0.246875,loss:3.509690
[epoch:0,batch:119]:acc: 0.299740,loss:3.279542
[epoch:0,batch:149]:acc: 0.340000,loss:3.105226
[epoch:0,batch:179]:acc: 0.371528,loss:2.942487
[epoch:0,batch:209]:acc: 0.401935,loss:2.796054
[epoch:0,batch:239]:acc: 0.422266,loss:2.690154
[epoch:0,batch:269]:acc: 0.440856,loss:2.587785
[epoch:0,batch:299]:acc: 0.456250,loss:2.500065
[epoch:0,batch:299]: val_loss:1.505650,val_acc:0.652126,val_total:4539
[epoch:0,batch:329]:acc: 0.466477,loss:2.426651
[epoch:0,batch:359]:acc: 0.476997,loss:2.360043
[epoch:0,batch:389]:acc: 0.487660,loss:2.295033
[epoch:0,batch:419]:acc: 0.498363,loss:2.236728
[epoch:0,batch:449]:acc: 0.504167,loss:2.190358
[epoch:0,batch:479]:acc: 0.509701,loss:2.151367
[epoch:0,batch:509]:acc: 0.516912,loss:2.111625
[epoch:0,batch:539]:acc: 0.522627,loss:2.07489

[epoch:4,batch:89]:acc: 0.826389,loss:0.468957
[epoch:4,batch:119]:acc: 0.820573,loss:0.485902
[epoch:4,batch:149]:acc: 0.816250,loss:0.486111
[epoch:4,batch:179]:acc: 0.816840,loss:0.484819
[epoch:4,batch:209]:acc: 0.814137,loss:0.489679
[epoch:4,batch:239]:acc: 0.814844,loss:0.482803
[epoch:4,batch:269]:acc: 0.813426,loss:0.484627
[epoch:4,batch:299]:acc: 0.811771,loss:0.488163
[epoch:4,batch:299]: val_loss:0.487751,val_acc:0.823309,val_total:4539
[epoch:4,batch:329]:acc: 0.811269,loss:0.490298
[epoch:4,batch:359]:acc: 0.809201,loss:0.493789
[epoch:4,batch:389]:acc: 0.808173,loss:0.492869
[epoch:4,batch:419]:acc: 0.808259,loss:0.491687
[epoch:4,batch:449]:acc: 0.807639,loss:0.492044
[epoch:4,batch:479]:acc: 0.808594,loss:0.488945
[epoch:4,batch:509]:acc: 0.809865,loss:0.486640
[epoch:4,batch:539]:acc: 0.810475,loss:0.484796
[epoch:4,batch:569]:acc: 0.811513,loss:0.485277
[epoch:4,batch:599]:acc: 0.811354,loss:0.487266
[epoch:4,batch:599]: val_loss:0.452308,val_acc:0.842036,val_total:

[epoch:8,batch:179]:acc: 0.866146,loss:0.330260
[epoch:8,batch:209]:acc: 0.865179,loss:0.332653
[epoch:8,batch:239]:acc: 0.863802,loss:0.335374
[epoch:8,batch:269]:acc: 0.864931,loss:0.329194
[epoch:8,batch:299]:acc: 0.863750,loss:0.332480
[epoch:8,batch:299]: val_loss:0.421895,val_acc:0.844018,val_total:4539
[epoch:8,batch:329]:acc: 0.862879,loss:0.334995
[epoch:8,batch:359]:acc: 0.861806,loss:0.336820
[epoch:8,batch:389]:acc: 0.862821,loss:0.334192
[epoch:8,batch:419]:acc: 0.863318,loss:0.334783
[epoch:8,batch:449]:acc: 0.863750,loss:0.333529
[epoch:8,batch:479]:acc: 0.863672,loss:0.333742
[epoch:8,batch:509]:acc: 0.862500,loss:0.336674
[epoch:8,batch:539]:acc: 0.861863,loss:0.337482
[epoch:8,batch:569]:acc: 0.861787,loss:0.337558
[epoch:8,batch:599]:acc: 0.861875,loss:0.336719
[epoch:8,batch:599]: val_loss:0.406975,val_acc:0.852390,val_total:4539
[epoch:8,batch:629]:acc: 0.862599,loss:0.335430
[epoch:8,batch:659]:acc: 0.862027,loss:0.338158
[epoch:8,batch:689]:acc: 0.860553,loss:0.3

[epoch:12,batch:269]:acc: 0.882292,loss:0.272554
[epoch:12,batch:299]:acc: 0.880833,loss:0.272900
[epoch:12,batch:299]: val_loss:0.405588,val_acc:0.853712,val_total:4539
[epoch:12,batch:329]:acc: 0.881345,loss:0.274671
[epoch:12,batch:359]:acc: 0.882205,loss:0.273312
[epoch:12,batch:389]:acc: 0.882372,loss:0.274121
[epoch:12,batch:419]:acc: 0.883110,loss:0.273723
[epoch:12,batch:449]:acc: 0.883333,loss:0.273373
[epoch:12,batch:479]:acc: 0.883398,loss:0.274222
[epoch:12,batch:509]:acc: 0.883578,loss:0.274083
[epoch:12,batch:539]:acc: 0.883275,loss:0.273001
[epoch:12,batch:569]:acc: 0.882456,loss:0.273727
[epoch:12,batch:599]:acc: 0.882760,loss:0.275466
[epoch:12,batch:599]: val_loss:0.404142,val_acc:0.856576,val_total:4539
[epoch:12,batch:629]:acc: 0.883383,loss:0.274488
[epoch:12,batch:659]:acc: 0.882860,loss:0.274877
[epoch:12,batch:689]:acc: 0.881975,loss:0.276958
[epoch:12,batch:719]:acc: 0.882118,loss:0.277353
[epoch:12,batch:749]:acc: 0.882042,loss:0.278993
[epoch:12,batch:779]:ac

[epoch:16,batch:329]:acc: 0.886080,loss:0.270472
[epoch:16,batch:359]:acc: 0.884896,loss:0.271295
[epoch:16,batch:389]:acc: 0.884455,loss:0.272247
[epoch:16,batch:419]:acc: 0.884226,loss:0.270969
[epoch:16,batch:449]:acc: 0.884653,loss:0.270167
[epoch:16,batch:479]:acc: 0.884375,loss:0.271685
[epoch:16,batch:509]:acc: 0.884804,loss:0.270916
[epoch:16,batch:539]:acc: 0.885012,loss:0.270483
[epoch:16,batch:569]:acc: 0.884923,loss:0.270051
[epoch:16,batch:599]:acc: 0.885000,loss:0.270463
[epoch:16,batch:599]: val_loss:0.401665,val_acc:0.857898,val_total:4539
[epoch:16,batch:629]:acc: 0.884325,loss:0.272445
[epoch:16,batch:659]:acc: 0.884754,loss:0.271665
[epoch:16,batch:689]:acc: 0.884239,loss:0.271983
[epoch:16,batch:719]:acc: 0.884852,loss:0.270887
[epoch:16,batch:749]:acc: 0.884625,loss:0.272414
[epoch:16,batch:779]:acc: 0.884415,loss:0.272090
[epoch:16,batch:809]:acc: 0.884375,loss:0.271081
[epoch:16,batch:839]:acc: 0.884487,loss:0.270671
[epoch:16,batch:869]:acc: 0.884950,loss:0.2695

In [4]:
def TrainWithRawData(path,epochNum):
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess(normalize_05,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_05,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    criterion=nn.CrossEntropyLoss().cuda()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])
    min_loss=modelParams['val_loss']
    print('min_loss is :%f'%(min_loss))
    print('val_correct is %f'%(modelParams['val_correct']))
    min_acc=max(modelParams['val_correct'],0.81)
    optinizerSave=modelParams['optimizer']
    patience=0
    lr=1e-4
    momentum=0.9
    beginepoch=modelParams['epoch']
    for epoch in range(beginepoch,epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if patience==3:
            patience=0
            model.load_state_dict(torch.load('../model/InceptionV3/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/5
            print('lr desencd')
        if epoch==beginepoch:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
#             optimizer.load_state_dict(optinizerSave)
#             lr=optimizer['lr']
#             momentum=optimizer['momentum']
            print('begin lr is ',lr)
            
        else:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
                   
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
                    if  log_loss < min_loss:
                        utils.snapshot('../model/', 'InceptionV3', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy })          

                        min_loss=log_loss
                        patience=0
                        print('save new model loss,now loss is ',min_loss)

                    if accuracy>min_acc:
                        utils.snapshot('../model/', 'InceptionV3', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy },key='acc') 
                        min_acc=accuracy
                        print('save new model acc,now acc is ',min_acc)
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))         
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'InceptionV3', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'InceptionV3', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
TrainWithRawData('../model/InceptionV3/2018-11-03_acc_best.pth',60)

[+] loading model... Done
min_loss is :0.403395
val_correct is 0.857678
Epoch 16/59
----------
begin lr is  0.0001
[epoch:16,batch:29]:acc: 0.881250,loss:0.267384
[epoch:16,batch:59]:acc: 0.903646,loss:0.232065
[epoch:16,batch:89]:acc: 0.911111,loss:0.216546
[epoch:16,batch:119]:acc: 0.910937,loss:0.217283
[epoch:16,batch:149]:acc: 0.907708,loss:0.219685
[epoch:16,batch:179]:acc: 0.907639,loss:0.223879
[epoch:16,batch:209]:acc: 0.907887,loss:0.222966
[epoch:16,batch:239]:acc: 0.908854,loss:0.222858
[epoch:16,batch:269]:acc: 0.907176,loss:0.226573
[epoch:16,batch:299]:acc: 0.908229,loss:0.225940
[epoch:16,batch:299]: val_loss:0.365122,val_acc:0.859220,val_total:4539
save new model loss,now loss is  0.3651220500469208
save new model acc,now acc is  tensor(0.8592, device='cuda:0')
[epoch:16,batch:329]:acc: 0.906250,loss:0.228048
[epoch:16,batch:359]:acc: 0.906424,loss:0.227143
[epoch:16,batch:389]:acc: 0.906891,loss:0.225844
[epoch:16,batch:419]:acc: 0.905878,loss:0.225585
[epoch:16,batch

[epoch:19,batch:959]:acc: 0.922266,loss:0.189728
[epoch:19,batch:989]:acc: 0.922317,loss:0.189579
[epoch:19] :acc: 0.922218,loss:0.189767,lr:0.000020,patience:0
[epoch:19]: val_loss:0.361762,val_acc:0.864067,
Epoch 20/59
----------
[epoch:20,batch:29]:acc: 0.898958,loss:0.244552
[epoch:20,batch:59]:acc: 0.909375,loss:0.232781
[epoch:20,batch:89]:acc: 0.912500,loss:0.219499
[epoch:20,batch:119]:acc: 0.909635,loss:0.223318
[epoch:20,batch:149]:acc: 0.907500,loss:0.225994
[epoch:20,batch:179]:acc: 0.905729,loss:0.225317
[epoch:20,batch:209]:acc: 0.903571,loss:0.228768
[epoch:20,batch:239]:acc: 0.906380,loss:0.224709
[epoch:20,batch:269]:acc: 0.905440,loss:0.227514
[epoch:20,batch:299]:acc: 0.903542,loss:0.230220
[epoch:20,batch:299]: val_loss:0.362256,val_acc:0.862304,val_total:4539
[epoch:20,batch:329]:acc: 0.902178,loss:0.232401
[epoch:20,batch:359]:acc: 0.902257,loss:0.232799
[epoch:20,batch:389]:acc: 0.901522,loss:0.234222
[epoch:20,batch:419]:acc: 0.901265,loss:0.234360
[epoch:20,bat

[epoch:23]: val_loss:0.359866,val_acc:0.865169,
Epoch 24/59
----------
lr desencd
[epoch:24,batch:29]:acc: 0.917708,loss:0.197088
[epoch:24,batch:59]:acc: 0.921875,loss:0.192556
[epoch:24,batch:89]:acc: 0.923958,loss:0.190294
[epoch:24,batch:119]:acc: 0.925260,loss:0.185550
[epoch:24,batch:149]:acc: 0.924375,loss:0.185052
[epoch:24,batch:179]:acc: 0.925521,loss:0.183981
[epoch:24,batch:209]:acc: 0.924851,loss:0.184335
[epoch:24,batch:239]:acc: 0.924219,loss:0.185699
[epoch:24,batch:269]:acc: 0.924769,loss:0.184526
[epoch:24,batch:299]:acc: 0.925208,loss:0.184945
[epoch:24,batch:299]: val_loss:0.363393,val_acc:0.863406,val_total:4539
[epoch:24,batch:329]:acc: 0.925852,loss:0.184372
[epoch:24,batch:359]:acc: 0.925955,loss:0.184110
[epoch:24,batch:389]:acc: 0.925721,loss:0.183978
[epoch:24,batch:419]:acc: 0.925223,loss:0.184706
[epoch:24,batch:449]:acc: 0.924653,loss:0.185568
[epoch:24,batch:479]:acc: 0.924674,loss:0.186215
[epoch:24,batch:509]:acc: 0.924203,loss:0.188180
[epoch:24,batch:

[epoch:28,batch:209]:acc: 0.902679,loss:0.226547
[epoch:28,batch:239]:acc: 0.902214,loss:0.225037
[epoch:28,batch:269]:acc: 0.902546,loss:0.224629
[epoch:28,batch:299]:acc: 0.902083,loss:0.224707
[epoch:28,batch:299]: val_loss:0.362158,val_acc:0.865389,val_total:4539
[epoch:28,batch:329]:acc: 0.903409,loss:0.223529
[epoch:28,batch:359]:acc: 0.904253,loss:0.222236
[epoch:28,batch:389]:acc: 0.904006,loss:0.223156
[epoch:28,batch:419]:acc: 0.904167,loss:0.222368
[epoch:28,batch:449]:acc: 0.904514,loss:0.221660
[epoch:28,batch:479]:acc: 0.903581,loss:0.223555
[epoch:28,batch:509]:acc: 0.903922,loss:0.223840
[epoch:28,batch:539]:acc: 0.903877,loss:0.223895
[epoch:28,batch:569]:acc: 0.903070,loss:0.225349
[epoch:28,batch:599]:acc: 0.903177,loss:0.225888
[epoch:28,batch:599]: val_loss:0.361153,val_acc:0.862965,val_total:4539
[epoch:28,batch:629]:acc: 0.903323,loss:0.225351
[epoch:28,batch:659]:acc: 0.902509,loss:0.226235
[epoch:28,batch:689]:acc: 0.902219,loss:0.226720
[epoch:28,batch:719]:ac

[epoch:32,batch:389]:acc: 0.904728,loss:0.229077
[epoch:32,batch:419]:acc: 0.904911,loss:0.228349
[epoch:32,batch:449]:acc: 0.904444,loss:0.228199
[epoch:32,batch:479]:acc: 0.904232,loss:0.228036
[epoch:32,batch:509]:acc: 0.904657,loss:0.227740
[epoch:32,batch:539]:acc: 0.904398,loss:0.227630
[epoch:32,batch:569]:acc: 0.903838,loss:0.227817
[epoch:32,batch:599]:acc: 0.904427,loss:0.227396
[epoch:32,batch:599]: val_loss:0.361058,val_acc:0.864508,val_total:4539
[epoch:32,batch:629]:acc: 0.903919,loss:0.228571
[epoch:32,batch:659]:acc: 0.904403,loss:0.228206
[epoch:32,batch:689]:acc: 0.904212,loss:0.228284
[epoch:32,batch:719]:acc: 0.904384,loss:0.227982
[epoch:32,batch:749]:acc: 0.903542,loss:0.229471
[epoch:32,batch:779]:acc: 0.903285,loss:0.230208
[epoch:32,batch:809]:acc: 0.903395,loss:0.230083
[epoch:32,batch:839]:acc: 0.903385,loss:0.229810
[epoch:32,batch:869]:acc: 0.903592,loss:0.229240
[epoch:32,batch:899]:acc: 0.903472,loss:0.229716
[epoch:32,batch:899]: val_loss:0.361201,val_ac